In [262]:
from utils import show

Note written 2020-02-14: Happy VD!  Current way of creating parts with inheritable values
is very unpythonic.

after 
```python
@makePart
class Bolts:
    Fu = 825
```
'Bolts' isn't a class anymore but is an instance of class Parts.  That doesn't seen right.
 Even though class objects are instances of class 'type'.
 
Here are experiments on how to retain the classness.

In [243]:
isinstance(object,type)

True

In [266]:
#from Designer import Part, CMPart

class Part:
    
    def __init__(self,names=""):
        self.__names = [k.strip() for k in names.split(',') if k.strip()]
        
    def __enter__(self):
        """Add all attributes/values to the set of global variables.
        Save enough state so that they can be restored when the context
        manager exits."""
        if hasattr(self,'__saved'):
            raise Exception('Object already is a context manager. Cannot be one again.')
        dct = vars(self)
        _new = []                # save a list of newly added variables
        _old = {}                # remember values of those that already exist in ns.
        ns = get_ipython().user_ns  # get the ns for the user
        for k in self.__names:
            if not hasattr(self,k):
                raise KeyError('Invalid attribute: '+k)
            if k in ns:
                _old[k] = ns[k]
            else:
                _new.append(k)
            ns[k] = getattr(self,k)
        self.__saved = (_new,_old)
        return self
    
    def __exit__(self,*l):
        """When the context exits, restore the global values to what they
        were before entering."""
        _new,_old = self.__saved
        ns = get_ipython().user_ns  # get the ns for the user
        for k,v in _old.items():
            ns[k] = v              # restore old values
        for k in _new:
            del ns[k]              # or delete them if they were newly created
        del self.__saved
        return False              # to re-raise exceptions
    
    
    @classmethod
    def only(cls,names=''):
        newdct = {}
        dct = cls.__dict__
        for k in names.split(','):
            k = k.strip()
            if k and k in dct:
                newdct[k] = dct[k]
        newdct['__doc__'] = dct.get('__doc__')
        newname = 'Part_of_'+cls.__name__
        return type(newname,cls.__bases__,newdct)

def makePart(cls):
    """Returns an object of type Part from the class definition and class attributes
    of 'cls'.  Intended to be used as a decorator so we can use class definitions
    to build parts (syntactic sugar)."""
    dct = cls.__dict__
    bases = cls.__bases__
    if Part not in cls.mro():
        if len(bases) > 0 and bases[-1] is object:
            bases = bases[:-1] + (Part,) + bases[-1:]
        else:
            bases = bases + (Part,)
        newdct = {k:v for k,v in dct.items() if not k.startswith('__')}
        newdct['__doc__'] = dct.get('__doc__')
        return type(cls.__name__,bases,newdct)
    return cls

In [272]:
@makePart 
class Foo:
    "this is a Foo"
    A = 3
    B = 4
    C = 5
    D = 6
    E = 7

In [273]:
@makePart
class Bar(Foo):
    A = 10
    C = 20

In [274]:
Bar.mro()

[__main__.Bar, __main__.Foo, __main__.Part, object]

In [275]:
%show Foo.A, Foo.B, Foo.C, Bar.A, Bar.B, Bar.C

Foo.A = 3  
Foo.B = 4  
Foo.C = 5  
Bar.A = 10 
Bar.B = 4  
Bar.C = 20 


In [228]:
Part in Bar.mro()

True

In [256]:
@makePart
class Zap(Foo.only('B,D')):
    A = 0

In [258]:
%show Zap.A, Zap.B, Zap.D, Zap.E

Zap.A = 0   
Zap.B = 4   
Zap.D = 6   
Zap.E = ??? 


In [251]:
class X:
    XX = 4

In [252]:
X.mro()

[__main__.X, object]

In [253]:
@makePart
class Y(X):
    YY = 5

In [254]:
Y.mro()

[__main__.Y, __main__.X, __main__.Part, object]

In [255]:
Y.__bases__

(__main__.X, __main__.Part)

In [263]:
with Zap('A,B,D'):
    show('A,B,D')
    B = 10
    with Foo('A'):
        show('A,B,D')
%show A,B,D

A = 0 
B = 4 
D = 6 
A = 3  
B = 10 
D = 6  
A = ??? 
B = ??? 
D = ??? 
